In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
from shapely.geometry import Polygon
import buffer_utils

In [9]:
grngdf = gpd.read_file('data/sgf_greenspaces.shp')

# to calculate a buffer around the park coords, need to first project to Azimuthal Equidistant crs.
# here, we're getting the projection string, which we pass to the function that projects it to the right crs
# each row needs its own projection string generated, its own buffer calculated based on that projection, and it needs it
#   buffer polygon projected *back* to wgs84
grngdf['coordinates'] = list(zip(grngdf['lat'], grngdf['lon']))
projection_strings = grngdf['coordinates'].apply(buffer_utils.generate_aeqd_params)

projection_inputs = pd.Series(list(zip(grngdf['coordinates'], projection_strings))
                              , name='aeqd projection inputs')

# clarifying that the buffer thats returned has been projected *back* to wgs84
grngdf['wgs84_buffer'] = projection_inputs.apply(buffer_utils.get_buffer)
grngdf.geometry = grngdf['wgs84_buffer']

In [10]:
sgfgdf = gpd.read_file('data/sgf_block_groups.shp')

intptgdf =sgfgdf# create gdf from internal points on block data.
intpt_geom = gpd.points_from_xy(sgfgdf['intptlon'], sgfgdf['intptlat'])
intptgdf = gpd.GeoDataFrame(sgfgdf.drop(columns='geometry')
                             , geometry=intpt_geom
                             , crs='EPSG:4269')

In [11]:
# project buffer gdf to crs from missouri block group data
prj_grngdf = grngdf.to_crs(epsg=4269)

# doing a spatial join
# checking if internal points of city block groups are withing the buffer of a park
joindf = gpd.sjoin(intptgdf, prj_grngdf, op='within', how='left')
# block groups with NaN values for the park data (e.g. 'name') are in the buffer of 0 parks.
# the count returns 0 for the nans, so it works

park_count = joindf.groupby('geoid', as_index=True)['name'].count()
park_count.name='park_count'

# add park_count series to the blockgdf columns
sgfgdf = sgfgdf.merge(park_count, left_on='geoid', left_index=False, right_index=True)

In [12]:
sgfgdf.to_file('data/sgf_greenspaces_density.shp')